# Dimensión mensajero

In [195]:
from datetime import date
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import holidays
import yaml

### Conexión con la base de datos

In [196]:
with open('../config.yml', 'r') as f:
  config = yaml.safe_load(f)
  config_oltp = config['fuente']
  config_olap = config['bodega']

url_oltp = (f"{config_oltp['drivername']}://{config_oltp['user']}:{config_oltp['password']}@{config_oltp['host']}:"f"{config_oltp['port']}/{config_oltp['dbname']}")
url_olap = (f"{config_olap['drivername']}://{config_olap['user']}:{config_olap['password']}@{config_olap['host']}:"f"{config_olap['port']}/{config_olap['dbname']}")
#print(url_oltp)
#print(url_olap)
oltp_conn = create_engine(url_oltp)
olap_conn = create_engine(url_olap)

### Módulo de extracción

In [197]:
tabla_mensajero = pd.read_sql_table('clientes_mensajeroaquitoy', oltp_conn)
tabla_user = pd.read_sql_table('auth_user', oltp_conn)
tabla_ciudad = pd.read_sql_table('ciudad', oltp_conn)
tabla_departamento = pd.read_sql_table('departamento', oltp_conn)

dimension_mensajero = (tabla_mensajero[['id', 'user_id', 'activo', 'fecha_entrada', 'fecha_salida', 'salario', 'telefono', 'ciudad_operacion_id']]
).rename(columns={'id' : 'id_mensajero'})
#dim_cliente = pd.read_sql_tabla('cliente', oltp_conn)
#dim_sede.info()
dimension_mensajero.head()

#tabla_departamento.head()

,id_mensajero,user_id,activo,fecha_entrada,fecha_salida,salario,telefono,ciudad_operacion_id
0,1,2,True,NaT,NaT,NaN,310-300000,13.0
1,42,410,True,NaT,NaT,NaN,310-300000,1.0
2,48,447,True,2024-07-12,NaT,1300000.0,310-300000,1.0
3,41,400,True,NaT,NaT,NaN,310-300000,1.0
4,13,331,True,2021-11-08,NaT,1160000.0,310-300000,4.0


### Módulo de transformación

In [198]:
tabla_departamento.rename(columns={'nombre' : 'departamento'}, inplace=True)

tabla_ciudad = pd.merge(tabla_ciudad.rename(columns={'nombre' : 'ciudad'}), tabla_departamento, how='inner'
                        ).drop(columns=['departamento_id']) # organizar # reemplazo de las llaves foraneas de la tabla ciudad (elimina departamento_id)

dimension_mensajero = pd.merge(dimension_mensajero, tabla_ciudad, how='left', left_on='ciudad_operacion_id', right_on='ciudad_id'
                               ).drop(columns=['ciudad_operacion_id', 'ciudad_id'])

tabla_user.drop(columns=['password', 'is_superuser', 'is_staff', 'date_joined', 'last_login'], inplace=True)

dimension_mensajero = pd.merge(dimension_mensajero, tabla_user, how='inner', left_on='user_id', right_on='id' # ¿inner?
                               ).drop(columns=['user_id', 'id', 'activo', 'is_active', 'fecha_salida'])

dimension_mensajero['fecha_entrada'] = dimension_mensajero['fecha_entrada'].dt.date
dimension_mensajero.set_index('id_mensajero', inplace=True)
dimension_mensajero.sort_values(by=['id_mensajero']).head()


,fecha_entrada,salario,telefono,ciudad,departamento,username,first_name,last_name,email
id_mensajero,,,,,,,,,
1,NaT,NaN,310-300000,ACOPI YUMBO,VALLE DEL CAUCA,mensajero1,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com
2,NaT,NaN,310-300000,NaN,NaN,mensajero2,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com
3,2012-05-08,3000000.0,310-300000,CALI,VALLE DEL CAUCA,Biil-Gates,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com
4,2018-12-17,1160000.0,310-300000,CALI,VALLE DEL CAUCA,Lionel_messi,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com
5,2015-07-01,1160000.0,310-300000,NaN,NaN,James Rodriguez,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com


### Módulo de carga a la bodega de datos

In [199]:
try: # Realizar una consulta sencilla para verificar la conexión
    # Ejecutar una consulta para obtener el nombre de las tablas
    tables = pd.read_sql("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public';", olap_conn)
    #print(tables)
    print("Conexión establecida y consulta realizada con éxito.")
except Exception as e:
    print(f"Error al conectar a la base de datos: {e}")

Conexión establecida y consulta realizada con éxito.


In [200]:
try:
  dimension_mensajero.to_sql('dim_mensajero', olap_conn, if_exists='replace')
except Exception as e:
  print(f"Error al cargar datos: {e}")